In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import json
import joblib
from tensorflow.keras.models import load_model

In [3]:
# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True,max_num_hands=1,min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

In [5]:
def extract_landmarks_for_prediction(hand_landmarks):
    """Extract landmarks in the format needed for prediction."""
    return np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten()

In [7]:
def load_recognition_components(model_path='asl_recognition_model811.h5', 
                              scaler_path='scaler.save', 
                              mappings_path='label_mappings.json'):
    """Load and return all necessary components for gesture recognition."""
    print("Loading model and components...")
    
    # Load the model
    model = load_model(model_path)
    
    # Load the scaler
    scaler = joblib.load(scaler_path)
    
    # Load label mappings
    with open(mappings_path, 'r') as f:
        mappings = json.load(f)
        label_map = mappings['label_map']
        reverse_label_map = {int(k): v for k, v in mappings['reverse_label_map'].items()}
    
    print("Model and components loaded successfully!")
    print(f"Number of gesture classes: {len(label_map)}")
    print("\nAvailable gestures:")
    for gesture in sorted(label_map.keys()):
        print(f"- {gesture}")
        
    return model, scaler, label_map, reverse_label_map

In [9]:
def real_time_prediction_3d(model, scaler, reverse_label_map):
    """Optimized real-time gesture recognition with 3D landmark visualization."""
    cap = cv2.VideoCapture(0)
    
    # Optimize camera buffer size
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
    
    # Set up real-time hand detection with optimized parameters
    hands_video = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.5,  # Slightly reduced for better performance
        min_tracking_confidence=0.5,
        model_complexity=0  # Use lighter model for better performance
    )
    
    print("Starting real-time prediction... Press 'q' to quit.")
    
    # Create black background for 3D landmarks (pre-allocated)
    landmark_frame = np.zeros((400, 400, 3), dtype=np.uint8)
    
    # Pre-calculate landmark IDs positions
    id_offsets = [(10, 10)] * 21  # Offset for landmark ID text
    
    # Initialize prediction variables
    last_prediction = None
    last_confidence = 0.0
    prediction_cooldown = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)
        
        # Convert the image to RGB (required by MediaPipe)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the frame with MediaPipe
        results = hands_video.process(image_rgb)
        
        # Reset landmark frame
        landmark_frame.fill(0)
        
        # Update prediction every 5 frames
        should_predict = prediction_cooldown == 0
        
        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]  # We only use the first hand
            
            if should_predict:
                # Extract landmarks and make prediction
                hand_landmarks_array = extract_landmarks_for_prediction(hand_landmarks)
                input_data = scaler.transform(hand_landmarks_array.reshape(1, -1))
                input_data = input_data.reshape(1, 21, 3)
                prediction = model.predict(input_data, verbose=0)
                print("\nprinting here ======================= ================= ============ ",prediction)
                last_prediction = reverse_label_map[np.argmax(prediction)]
                print("\nprinting np.argmax(prediction) ======================= ================= ============ ",np.argmax(prediction))
                print("\nprinting last_prediction ======================= ================= ============ ",last_prediction)
                last_confidence = np.max(prediction)
                prediction_cooldown = 5  # Reset cooldown
            
            # Draw the hand landmarks on the original frame
            mp_drawing.draw_landmarks(
                frame, 
                hand_landmarks, 
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=1),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )
            
            # Efficient 3D landmark visualization
            landmarks = hand_landmarks.landmark
            frame_h, frame_w = landmark_frame.shape[:2]
            
            # Draw connections first (reduces overlapping)
            for connection in mp_hands.HAND_CONNECTIONS:
                start_pos = landmarks[connection[0]]
                end_pos = landmarks[connection[1]]
                
                start_x = int(start_pos.x * frame_w)
                start_y = int(start_pos.y * frame_h)
                end_x = int(end_pos.x * frame_w)
                end_y = int(end_pos.y * frame_h)
                
                # Use average z-depth for line color
                avg_z = ((start_pos.z + 1) / 2 + (end_pos.z + 1) / 2) / 2
                color_intensity = int(avg_z * 255)
                
                cv2.line(landmark_frame, (start_x, start_y), (end_x, end_y),
                        (0, color_intensity, 255-color_intensity), 1)
            
            # Draw landmarks
            for idx, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_w)
                y = int(landmark.y * frame_h)
                
                # Normalize z coordinate once
                z_normalized = (landmark.z + 1) / 2
                radius = int((1 - z_normalized) * 8) + 2
                color_intensity = int(z_normalized * 255)
                
                cv2.circle(landmark_frame, (x, y), radius, 
                         (0, color_intensity, 255-color_intensity), -1)
                
                # Add landmark ID
                cv2.putText(landmark_frame, str(idx), (x+10, y+10),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)
        
        # Decrement prediction cooldown
        if prediction_cooldown > 0:
            prediction_cooldown -= 1
        
        # Display prediction on original frame
        if last_prediction is not None:
            text = f"{last_prediction.upper()} ({last_confidence:.2f})"
            cv2.putText(frame, text, (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Add title to landmark frame (only once per frame)
        cv2.putText(landmark_frame, "3D Hand Landmarks", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(landmark_frame, "Color indicates Z-depth", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Show both frames
        cv2.imshow('Gesture Recognition', frame)
        cv2.imshow('3D Hand Landmarks', landmark_frame)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Clean up
    cap.release()
    cv2.destroyAllWindows()
    hands_video.close()

In [11]:
model, scaler, label_map, reverse_label_map = load_recognition_components(
    model_path='asl_recognition_model811.h5',
    scaler_path='scaler.save',
    mappings_path='label_mappings.json'
)

Loading model and components...
Model and components loaded successfully!
Number of gesture classes: 40

Available gestures:
- a
- b
- c
- call
- d
- del
- dislike
- e
- f
- fist
- g
- grabbing
- grip
- h
- i
- j
- k
- l
- like
- m
- n
- nothing
- o
- ok
- p
- palm
- q
- r
- rock
- s
- space
- stop
- stop_inv
- t
- u
- v
- w
- x
- y
- z


In [19]:
# For real-time prediction with 3d
try:
    real_time_prediction_3d(model, scaler, reverse_label_map)
except:
    print("Something else went wrong")
    cv2.destroyAllWindows()
finally:
    cv2.destroyAllWindows()

Starting real-time prediction... Press 'q' to quit.

printing here ======================= ================= ============  [[8.0551391e-17 0.0000000e+00 0.0000000e+00 1.7529662e-16 0.0000000e+00
  0.0000000e+00 1.8532174e-17 2.0117093e-25 3.9590404e-33 1.0000000e+00
  0.0000000e+00 1.3323870e-30 1.6610254e-22 0.0000000e+00 1.0075484e-20
  2.7578606e-20 4.6698038e-33 0.0000000e+00 1.0253884e-20 4.2034129e-17
  5.3079314e-20 0.0000000e+00 0.0000000e+00 0.0000000e+00 3.0564608e-27
  0.0000000e+00 0.0000000e+00 1.0691685e-26 8.2435821e-17 3.5984175e-16
  0.0000000e+00 0.0000000e+00 0.0000000e+00 1.3529179e-18 0.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0331665e-20 1.0591688e-21 1.3767806e-15]]

printing np.argmax(prediction) ======================= ================= ============  9

printing last_prediction ======================= ================= ============  fist


C:\Users\abhis\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\abhis\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



printing here ======================= ================= ============  [[2.43833137e-18 0.00000000e+00 0.00000000e+00 1.00372586e-12
  0.00000000e+00 0.00000000e+00 2.18984986e-14 1.04349446e-24
  1.92494088e-31 1.00000000e+00 7.03067403e-31 5.49324091e-29
  1.14998938e-20 0.00000000e+00 9.66068787e-18 4.49426602e-14
  1.29676753e-33 0.00000000e+00 1.51654579e-15 1.20817516e-20
  8.65887556e-22 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.15117547e-22 0.00000000e+00 1.89665388e-37 4.44122921e-25
  3.20905801e-15 2.35523334e-15 6.06934733e-34 1.20423642e-35
  0.00000000e+00 4.93923858e-20 0.00000000e+00 0.00000000e+00
  0.00000000e+00 4.18324063e-25 4.21631058e-16 2.30048319e-21]]

printing np.argmax(prediction) ======================= ================= ============  9

printing last_prediction ======================= ================= ============  fist

printing here ======================= ================= ============  [[1.5067332e-20 0.0000000e+00 0.0000000e+00 1.5120330e-13 0

C:\Users\abhis\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



printing here ======================= ================= ============  [[4.9933230e-21 0.0000000e+00 0.0000000e+00 2.1367826e-13 0.0000000e+00
  0.0000000e+00 6.2137379e-16 9.0331147e-30 8.4912032e-37 1.0000000e+00
  9.9828180e-37 2.2499665e-33 3.7945720e-24 0.0000000e+00 9.2839139e-22
  4.1478701e-17 1.2212316e-37 0.0000000e+00 4.9923427e-17 8.6677236e-24
  2.2261404e-24 0.0000000e+00 0.0000000e+00 0.0000000e+00 3.2555491e-26
  0.0000000e+00 0.0000000e+00 1.3429657e-29 1.7143972e-17 1.0522838e-18
  0.0000000e+00 0.0000000e+00 0.0000000e+00 3.9505523e-23 0.0000000e+00
  0.0000000e+00 0.0000000e+00 1.1480473e-28 9.3060488e-20 1.9116307e-23]]

printing np.argmax(prediction) ======================= ================= ============  9

printing last_prediction ======================= ================= ============  fist


...


printing here ======================= ================= ============  [[3.1657196e-21 5.8878670e-22 2.7157086e-20 3.5415400e-02 1.1227737e-19
  5.0123870e-13 6.7739666e-02 2.5371012e-19 7.6115208e-18 1.9873016e-01
  2.3596825e-20 7.2927006e-02 2.6325420e-01 4.2815206e-13 3.3793598e-13
  2.1254181e-08 7.9223078e-19 6.7026561e-18 2.0029709e-02 8.6842605e-22
  9.6846274e-15 1.8165937e-38 8.3351525e-21 3.7901133e-02 1.5755335e-10
  8.8813175e-03 4.7480154e-20 1.5797640e-14 1.6928519e-01 5.5008028e-20
  7.1273836e-09 8.7729216e-02 3.8106944e-02 6.7374239e-16 1.8517272e-19
  8.5017562e-13 6.8678570e-20 4.2285434e-23 3.1762318e-13 2.1254661e-16]]

printing np.argmax(prediction) ======================= ================= ============  12

printing last_prediction ======================= ================= ============  grip


C:\Users\abhis\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



printing here ======================= ================= ============  [[2.64780721e-18 2.09302234e-18 3.03602570e-15 3.66593488e-02
  4.39062916e-15 2.88814439e-09 2.37648934e-01 2.17781792e-19
  2.75557437e-18 3.89412791e-02 8.17524894e-20 1.94066629e-01
  5.38120307e-02 3.19698776e-13 1.13664706e-13 6.38085210e-11
  1.24260051e-15 1.54520910e-11 7.44815692e-02 7.86170032e-20
  5.15277274e-12 2.99587854e-31 1.66900222e-18 1.84345227e-02
  4.61193944e-10 8.85790437e-02 1.65672948e-16 1.75349690e-16
  2.07905814e-01 3.71937062e-22 1.12453037e-07 4.01883386e-02
  9.28237662e-03 4.47815172e-14 6.74549444e-17 8.50936044e-10
  3.11376681e-15 1.12325185e-22 2.84250683e-12 2.96265779e-13]]

printing np.argmax(prediction) ======================= ================= ============  6

printing last_prediction ======================= ================= ============  dislike


C:\Users\abhis\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



printing here ======================= ================= ============  [[2.36600815e-14 8.30376282e-14 6.87072469e-13 1.12841234e-01
  1.40958230e-12 3.72925200e-07 1.07481495e-01 1.26587981e-14
  3.49820101e-14 1.13463774e-01 1.30655654e-14 1.37567163e-01
  7.87553117e-02 1.17487098e-09 2.46043325e-10 1.50755977e-06
  2.23155197e-12 7.63433539e-10 9.04959887e-02 1.85954585e-15
  1.36967315e-09 2.56544082e-23 1.86401035e-14 7.62210563e-02
  8.20490982e-08 5.06936908e-02 2.41656196e-13 2.68755374e-12
  9.72933248e-02 4.57021762e-16 5.44077966e-06 6.78974167e-02
  6.72820583e-02 2.53448981e-11 5.72554916e-13 2.90742168e-08
  4.05984578e-12 2.07145431e-17 1.57763913e-09 6.68223116e-11]]

printing np.argmax(prediction) ======================= ================= ============  11

printing last_prediction ======================= ================= ============  grabbing


<h3> Run it for 2d Testing</h3>

In [15]:
def extract_hand_landmarks(image):
    """Extract hand landmarks from an image."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw landmarks on the image
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Extract landmark coordinates
            landmark_array = np.array([[lm.x, lm.y, lm.z] 
                                     for lm in hand_landmarks.landmark])
            return landmark_array.flatten()
    return np.zeros(21 * 3)

In [17]:
def predict_gesture(image, model, scaler, reverse_label_map):
    """Predict gesture from an image."""
    # Extract landmarks
    hand_landmarks = extract_hand_landmarks(image)
    
    if np.any(hand_landmarks):
        # Preprocess the landmarks
        input_data = scaler.transform(hand_landmarks.reshape(1, -1))
        input_data = input_data.reshape(1, 21, 3)
        
        # Make prediction
        prediction = model.predict(input_data, verbose=0)
        predicted_class = np.argmax(prediction)
        confidence = np.max(prediction)
        
        # Get predicted label
        predicted_label = reverse_label_map[predicted_class]
        
        return predicted_label, confidence
    
    return None, 0.0

In [13]:
def real_time_prediction(model, scaler, reverse_label_map):
    """Perform real-time gesture recognition using webcam."""
    cap = cv2.VideoCapture(0)
    
    # Set up real-time hand detection
    hands_video = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.5
    )
    
    print("Starting real-time prediction... Press 'q' to quit.")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Flip the frame horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)
        
        # Make prediction
        predicted_label, confidence = predict_gesture(
            frame, model, scaler, reverse_label_map)
        
        # Display prediction
        if predicted_label is not None:
            text = f"{predicted_label.upper()} ({confidence:.2f})"
            cv2.putText(frame, text, (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Show the frame
        cv2.imshow('Real-time Gesture Recognition', frame)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Clean up
    cap.release()
    cv2.destroyAllWindows()
    hands_video.close()

In [ ]:
# For real-time prediction
try:
    real_time_prediction(model, scaler, reverse_label_map)
except:
    print("Something else went wrong")
    cv2.destroyAllWindows()
finally:
    cv2.destroyAllWindows()